In [4]:
import os
from typing import List
from pydantic import BaseModel, Field, AliasChoices, model_validator
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from constants import BASE_URL, API_KEY, MODEL_NAME

class Topic(BaseModel):
    title: str = Field(
        validation_alias=AliasChoices('title', 'topic_name', 'тема', 'заголовок'),
        description="Название темы"
    )
    description: str = Field(
        validation_alias=AliasChoices('description', 'preview', 'описание', 'summary'),
        description="О чем будет пост"
    )
    # Добавил 'reason', так как LLM выбрала именно его в вашем логе
    reasoning: str = Field(
        validation_alias=AliasChoices('reasoning', 'rationale', 'logic', 'обоснование', 'why', 'reason'),
        description="Почему эта тема актуальна сейчас"
    )

    @model_validator(mode='before')
    @classmethod
    def handle_nesting(cls, v):
        if isinstance(v, dict) and len(v) == 1 and any(k in v for k in ['topic', 'тема', 'item']):
            return list(v.values())[0]
        return v

class TopicList(BaseModel):
    suggestions: List[Topic] = Field(
        validation_alias=AliasChoices('suggestions', 'topics', 'items', 'темы', 'themes'),
        description="Список предложенных тем"
    )

class TopicStrategist:
    def __init__(self):
        self.llm = ChatOpenAI(
            base_url=BASE_URL, 
            api_key=API_KEY, 
            model=MODEL_NAME, 
            temperature=0.8 # Чуть выше для креативности
        )
        self.data_dir = "../DATA/"

    def _load_context(self) -> dict:
        """Загружает базу знаний для анализа."""
        def read(name):
            path = os.path.join(self.data_dir, name)
            return open(path, "r", encoding="utf-8").read() if os.path.exists(path) else ""
        
        return {
            "archive": read("posts.md"),
            "objectives": read("post_requirements.md")
        }

    def get_suggestions(self, k: int = 5) -> List[Topic]:
        """Основной метод для получения идей."""
        ctx = self._load_context()
        structured_llm = self.llm.with_structured_output(TopicList, method="json_mode")
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", (
                "Ты — контент-стратег для Telegram-канала.\n"
                "Твоя цель: изучить архив постов и выявить 'белые пятна' (темы, которые важны, но еще не освещались).\n"
                "Предложения должны быть свежими, не повторять архив и бить точно в цели автора.\n"
                "Отвечай СТРОГО в формате JSON с ключом 'suggestions'."
            )),
            ("human", (
                "АРХИВ ПОСТОВ:\n{archive}\n\n"
                "ЦЕЛИ КАНАЛА:\n{objectives}\n\n"
                "ЗАДАНИЕ: Предложи {k} новых тем. Для каждой укажи заголовок, краткое описание и "
                "обоснуй, почему именно эта тема сейчас нужна аудитории."
            ))
        ])
        
        chain = prompt | structured_llm
        
        # Передаем контекст (с ограничением архива, чтобы не переполнить память)
        result = chain.invoke({
            "archive": ctx['archive'][:6000], 
            "objectives": ctx['objectives'], 
            "k": k
        })
        return result.suggestions

strategist = TopicStrategist()

k_input = input("Сколько идей сгенерировать? (по умолчанию 5): ")
k = int(k_input) if k_input.isdigit() else 5

print(f"\n🚀 Ищу свежие идеи для канала...")
ideas = strategist.get_suggestions(k=k)

print("\n" + "="*50)
print(f"ТОП-{len(ideas)} ТЕМ ДЛЯ БУДУЩИХ ПОСТОВ")
print("="*50)

for i, idea in enumerate(ideas, 1):
    print(f"{i}. 💡 {idea.title.upper()}")
    print(f"   📝 Суть: {idea.description}")
    print(f"   🎯 Почему это сработает: {idea.reasoning}")
    print("-" * 50)


🚀 Ищу свежие идеи для канала...

ТОП-10 ТЕМ ДЛЯ БУДУЩИХ ПОСТОВ
1. 💡 КАК НАЙТИ РАБОТУ В КИТАЕ ПОСЛЕ УЧЕБЫ: ЛИЧНЫЙ ОПЫТ
   📝 Суть: Рассказ о том, как искать работу в Китае, какие есть особенности местного рынка труда, и какие сложности могут возникнуть у иностранцев. Обзор популярных платформ для поиска работы и советы по составлению резюме на китайский манер.
   🎯 Почему это сработает: Аудитория интересуется не только учебой, но и карьерными перспективами. Эта тема поможет тем, кто рассматривает возможность остаться в Китае после окончания учебы.
--------------------------------------------------
2. 💡 КУЛЬТУРНЫЕ РАЗЛИЧИЯ: ЧТО УДИВИЛО В КИТАЕ БОЛЬШЕ ВСЕГО
   📝 Суть: Обзор неочевидных культурных особенностей Китая, которые могут удивить иностранца. Например, отношение к времени, личному пространству, или традиции, которые не всегда понятны с первого взгляда.
   🎯 Почему это сработает: Посты про жизнь в Китае пользуются популярностью, и эта тема дополнит серию, добавив новый угол зрения н